In [8]:
import pandas as pd 
import zipfile

zipfile_path = '/Users/lenoxherman/Tippers/archive (1).zip'


# Extract the CSV file from the zip archive
with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
    # Assuming the CSV file is directly in the root of the zip file
    tip_data = zip_ref.namelist()[0]  # Assuming it's the first file in the list
    zip_ref.extract(tip_data, path='.')

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(tip_data)



FileNotFoundError: [Errno 2] No such file or directory: '/Users/lenoxherman/Tippers/archive.zip'